# 1.11.5 Tuning and Optimization in *Testing, Debugging, Profiling, and Tuning*

In [32]:
%reload_ext line_profiler  # used to profile Python Code 

## Tuning Strategies

### Understand Your Program (1. Uderstand Algorithms 2. Use the Built-In Types)

In [5]:
import collections
s1 = collections.deque()
s1.appendleft(37)
s1.appendleft(38)
print s1

s2 = []
s2.insert(0,37)
s2.insert(0,38)
print s2

deque([38, 37])
[38, 37]


In [3]:
# Before 
timeit('s.insert(0,37)','s = []', number=1000000)

376.56451082229614

In [2]:
# After 
from timeit import timeit
timeit('s.appendleft(37)', 'import collections; s = collections.deque()', number=1000000)

0.16429996490478516

### Don't Add Layers

In [57]:
%timeit s = dict(name='Good', shares=100, price=490.10)  # because it adds an extra function call

The slowest run took 6.72 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 284 ns per loop


In [60]:
%timeit s = {'name':'Good', 'shares':100, 'price': 490.10}

1000000 loops, best of 3: 139 ns per loop


### Know How Classes and Instance Build Upon Dictionaries

In [62]:
# User-defined classes and instances are built using dictionaries.
# Because of this, operations that look up, set, or delete instance data are almost always going to run more
# slowly than directly performing these operations on a dictioanry.

class Stock(object):
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [66]:
%timeit s1 = Stock('Good', 100, 490.10)

The slowest run took 10.44 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 571 ns per loop


In [79]:
%timeit s2 = {'name':'Good', 'shares':100, 'price': 490.10}

1000000 loops, best of 3: 146 ns per loop


In [68]:
%timeit s1.shares*s1.price

The slowest run took 13.43 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 160 ns per loop


In [81]:
s2 = {'name':'Good', 'shares':100, 'price': 490.10}

In [84]:
%timeit s2['shares']*s2['price']

The slowest run took 20.91 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 103 ns per loop


### Use __slots__

In [ ]:
# if your program creates a large number of instances of user-defined classes,
# you might consider using the __slots__ atribute in a class definition

class Stock(object):
    __slot__ = ['name','shares','price']
    def __init__(self, name, shares, price):
        self.name = name 
        self.shares = shares
        self.price = price

### Avoid the (.) Operator

In [91]:
import math

In [92]:
%timeit math.sqrt(16) 
# becuase whenever you use the (.) to look up an attribute on an object, it always involves a name lookup.
# However, it makes your code easy to read.

The slowest run took 20.22 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 94.4 ns per loop


In [87]:
from math import sqrt 

In [89]:
%timeit sqrt(16)

The slowest run took 31.62 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 67.9 ns per loop


### Use Exceptions to Handle Uncommon Cases

In [49]:
line = "Apple iPhone X: Fix For Cold Weather Problem Coming Soon"

In [50]:
# Before 
def parse_header1(line):
    fields = line.split(":")
    if len(fields) != 2:
        raise RuntimeError("Malformed header")
    header, value = fields
    return header.lower(), value.strip()

In [51]:
%timeit parse_header1(line)

The slowest run took 5.29 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 946 ns per loop


In [55]:
%lprun -f parse_header1 parse_header1(line)

Timer unit: 1e-06 s

Total time: 6e-06 s
File: <ipython-input-50-382e7ea91819>
Function: parse_header1 at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def parse_header1(line):
     3         1            3      3.0     50.0      fields = line.split(":")
     4         1            1      1.0     16.7      if len(fields) != 2:
     5                                                   raise RuntimeError("Malformed header")
     6         1            0      0.0      0.0      header, value = fields
     7         1            2      2.0     33.3      return header.lower(), value.strip()

In [52]:
# After 
def parse_header2(line):
    fields = line.split(":")
    try:
        header, value = fields
        return header.lower(), value.strip()
    except ValueError:
        raise RuntimeError("Malformed header")

In [53]:
%timeit parse_header2(line)

The slowest run took 4.40 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 922 ns per loop


In [56]:
%lprun -f parse_header2 parse_header2(line)

Timer unit: 1e-06 s

Total time: 9e-06 s
File: <ipython-input-52-9e2dea3949a9>
Function: parse_header2 at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def parse_header2(line):
     3         1            4      4.0     44.4      fields = line.split(":")
     4         1            1      1.0     11.1      try:
     5         1            1      1.0     11.1          header, value = fields
     6         1            3      3.0     33.3          return header.lower(), value.strip()
     7                                               except ValueError:
     8                                                   raise RuntimeError("Malformed header")

### Avoid Exceptions for Common Cases

In [ ]:
# suppose you had a program that performed a lot of dictionary lookups, 
# but most of these lookups were keys that didn't exist 

# Before
try:
    value = items[key]
except KeyError:
    value = None 

In [ ]:
# After: 17 times faster
if key in items:
    value = items[key]
else:
    value = None 

# Besides, this latter approach also runs almost twice as fast as using items.get(key)
# Because the in operator is faster to execute than a method call

### Embrace Functional Programming and Iteration 

In [104]:
before = []

In [105]:
%timeit for i in range(10): before.append(i)

The slowest run took 5.23 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 1.32 µs per loop


In [106]:
%timeit after = [i for i in range(10)] # runs fast, makes efficient use of memory

100000 loops, best of 3: 1.21 µs per loop


### Use Decorators and Metaclasses

## Reference

- http://mortada.net/easily-profile-python-code-in-jupyter.html
- https://stackoverflow.com/questions/22108488/are-list-comprehensions-and-functional-functions-faster-than-for-loops